In [2]:
from settings import *
import soundfile as sf
import numpy as np

In [3]:
normal_boundary = 0.03
qt_boundary = 0.0001
qs_boundary = 0.08

# Load all the signals in the numbers folder and cut them if the amplitude is too low and save them in a new folder
for number in NUMBERS:
    for person in PEOPLE:
        audio_path = NUMBERS_FOLDER + "/" + person + "_" + number + AUDIO_FORMAT
        num_audio, sample_rate = sf.read(audio_path)

        # find the first occurrence of an array elemen  t that is greater than boundary
        # find the last occurrence of an array element that is greater than boundary

        # if person == "qt" or person == "qh":
        #     num_audio = num_audio[np.where(abs(num_audio) >= qt_boundary)[0][0]:np.where(abs(num_audio) >= qt_boundary)[0][-1]-800]    
        
        # if person == "qs":
        #     num_audio = num_audio[np.where(abs(num_audio) >= qs_boundary)[0][0]:np.where(abs(num_audio) >= qs_boundary)[0][-1]-200]    
        
        # else:
        num_audio = num_audio[np.where(abs(num_audio) >= normal_boundary)[0][0]:np.where(abs(num_audio) >= normal_boundary)[0][-1]-1000]    
        
        sf.write("./processed_audio/" + person + "_" + number + "_cut.wav", num_audio, sample_rate)